<a href="https://colab.research.google.com/github/chunyulin/tf2020ncku/blob/master/2_keras_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reuse models: Keras Application
* https://keras.io/api/applications/

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

print("X:", x_train.shape)
print("Y:", y_train.shape)

X: (60000, 28, 28, 1)
Y: (60000,)


In [0]:
# Load data with `tf.data.Dataset`
def enlarge(x,y):
  x = tf.reshape(x, (-1,1))
  x = tf.tile(x,  tf.constant([1,3], tf.int32) )
  x = tf.reshape(x, (28,28,3))            ## duplicate 3 channel
  largex = tf.image.resize(x, [32, 32])   ##  denseNet need image >= 32x32
  return largex, y

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(enlarge).shuffle(10000).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
test_ds  = tf.data.Dataset.from_tensor_slices((x_test, y_test)).map(enlarge).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [3]:
for x,y in test_ds.take(1):
  print(x.shape)

(64, 32, 32, 3)


# Use DenseNet for MNIST (just for fun)
* [Comparasion of models](https://www.cnblogs.com/shine-lee/p/11426134.html)

![Comparasion of models](https://miro.medium.com/max/658/1*XoakexX4n9YSEalWxePqqw.png)
* See also the [report on news](https://kknews.cc/zh-tw/tech/nekyn68.html) or the [original paper](https://arxiv.org/abs/1801.04381)

In [0]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model

def add_last_layer(base_model, nb_classes):
  """
  Add one layer for model to classify nb_class class
  """
  x = base_model.output
  predict = Dense(nb_classes, activation='softmax')(x)
  return Model(inputs=base_model.input, outputs=predict)

model0 = tf.keras.applications.densenet.DenseNet121(include_top=False, input_shape=(32,32,3))
model = add_last_layer(model0, 10)
#tf.keras.backend.image_data_format()

In [0]:
model.summary()

In [0]:
# Define loss and optimzer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## define loss/acc fucntion
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## define train/test function
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [0]:
EPOCHS = 5

train_loss_history = []
test_loss_history = []

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  train_loss_history.append(np.mean(train_loss.result()))
  test_loss_history.append(np.mean(test_loss.result()))

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

  # Reset the metrics for the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.27818572521209717, Accuracy: 11.29642391204834, Test Loss: 0.08905678987503052, Test Accuracy: 11.059990882873535
Epoch 2, Loss: 0.06764649599790573, Accuracy: 11.406426429748535, Test Loss: 0.04779548570513725, Test Accuracy: 11.057957649230957
Epoch 3, Loss: 0.05059191584587097, Accuracy: 11.428725242614746, Test Loss: 0.032907333225011826, Test Accuracy: 11.062963485717773
Epoch 4, Loss: 0.046368468552827835, Accuracy: 11.43455982208252, Test Loss: 0.0639394074678421, Test Accuracy: 11.040592193603516


# Let's make prediction with the model

In [0]:
import matplotlib.pyplot as plt

for images,labels in test_ds.take(2):
  print(images.shape)  ## (64, 32, 32, 3)
  im = images[7:8]     ## take 6th image out of the batch.
  print("Predict: ", model.predict(im).shape)
  #plt.imshow(im.numpy().reshape(32,32))
  #plt.show()


(64, 32, 32, 3)
Predict:  (1, 1, 1, 1024)
(64, 32, 32, 3)
Predict:  (1, 1, 1, 1024)


In [0]:
# Make prediction via `model.predict()` for multimple sample
for images,labels in test_ds.take(1):
  pre = model.predict(images).argmax(axis=1)
  print( pre)
  print( labels)
  print( pre-labels.numpy())
  

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1 3 4 7 2 7
 1 2 1 1 7 4 2 3 5 1 2 4 4 6 3 5 5 6 0 4 1 9 5 7 8 9 3]
tf.Tensor(
[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4 9 6 6 5 4 0 7 4 0 1 3 1 3 4 7 2 7
 1 2 1 1 7 4 2 3 5 1 2 4 4 6 3 5 5 6 0 4 1 9 5 7 8 9 3], shape=(64,), dtype=uint8)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
